In [ ]:
import cv2
import random
import numpy as np

In [ ]:
import zipfile
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/

Mounted at /content/drive
/content/drive


In [ ]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/dataset.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0
test_batch_pointer = 0

#read data.txt
with open("/tmp/dataset/driving_log.csv") as f:
    for line in f:
        xs.append("/tmp/dataset/IMG/" + line.split(',')[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split(',')[1]) * 3.14159265 / 180)

#get number of images
num_images = len(xs)

#shuffle list of images
c = list(zip(xs, ys))
random.shuffle(c)
xs, ys = zip(*c)

train_xs = xs[:int(len(xs) * 0.8)]
train_ys = ys[:int(len(xs) * 0.8)]

val_xset = xs[-int(len(xs) * 0.2):]
val_yset = ys[-int(len(xs) * 0.2):]

val_xs = val_xset[:int(len(val_xset) * 0.5)]
val_ys = val_yset[:int(len(val_yset) * 0.5)]

test_xs = val_xset[-int(len(val_xset) * 0.5):]
test_ys = val_yset[-int(len(val_yset) * 0.5):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)
num_test_images = len(test_xs)

In [ ]:
num_train_images

25954

In [ ]:
num_val_images

3244

In [ ]:
num_test_images

3244

In [ ]:
def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
      x_out.append(cv2.resize(cv2.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], (200, 66)) / 255.0)
      y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
      x_out.append(cv2.resize(cv2.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], (200, 66)) / 255.0)
      y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out

def LoadTestBatch(batch_size):
    global test_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
      x_out.append(cv2.resize(cv2.imread(test_xs[(test_batch_pointer + i) % num_test_images])[-150:], (200, 66)) / 255.0)
      y_out.append([test_ys[(test_batch_pointer + i) % num_test_images]])
    test_batch_pointer += batch_size
    return x_out, y_out